# 설명
Json Data를 MySQL Query로 변경해주는 파일

In [22]:
from util.file_handler import iterator_dir_children, make_file_path, load_json

In [23]:
# 특정 경로 내부의 모든 파일을 Generator로 가져오기.
SOURCE_DIR = "wanted_json"
SQL_FILE = "wanted_mysql.sql"

json_paths = iterator_dir_children(SOURCE_DIR)

In [24]:
def extract_json(file_name):
    return load_json(file_name)

In [25]:
def get_company_name(json):
    if "company_name" not in json: return ""
    return json["company_name"]

def get_location(json):
    if "location" not in json: return ""
    return json["location"]

def get_sales(json):
    if "sales" not in json: return ""
    return json["sales"]

In [26]:
query_insert = "insert into company (company_name, location, sales) VALUES"
query_data = lambda json: f"('{get_company_name(json)}', '{get_location(json)}', {get_sales(json)})"

In [27]:
def first_query(query):
    return f"{query_insert} {query}"

def other_query(query):
    return f",{query}"

def make_query(generator):
    yield first_query(next(generator))
    
    for query in generator:
        yield other_query(query)
    
    yield ";"

In [28]:
file_path = make_file_path(SQL_FILE)

with open(file_path, 'w', encoding="utf-8") as file:
    json_iterator = map(extract_json, json_paths)
    query_iterator = map(query_data, json_iterator)
    
    for query in make_query(query_iterator):
        file.write(query)
